In [22]:
import os
import sys
import re
import random
import numpy as np
import pandas as pd
import math
import glob
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
pd.options.display.max_colwidth = 1000

sys.path.append('/home/linnaan/lib/')
from libSlurm import make_submit_file

sys.path.append('/home/linnaan/software/')
from silent_tools import silent_tools

In [2]:
original_seq = 'NEDIRRLLHEFYEALNRGDLDKAMELYHPAGRLTVNGREYHSRDEIREFLKHAIDANVRFEIDHIEIRGETVTVHATVHVRGGNLTFRVTEHWVFDPNNRVVEVHIHIRPP'

In [11]:
script_dir = '/home/linnaan/software/analyze_ssm_bcov/'
work_dir = '/home/linnaan/wet_lab/NGS/FOL-SSM-1739/ssm_analysis_all/'

In [23]:
PROTOCOL = f'/software/rosetta/latest/bin/rosetta_scripts.hdf5.linuxgccrelease -corrections::beta_nov16 ' + \
           f'-parser:protocol {script_dir}ssm_relax_then_score.xml -nstruct 3'
param = '/home/linnaan/ligands/FOL/FOL_classic_redo/FOL.params'
out = f'{work_dir}relax_all_silent_out'
os.makedirs(out,exist_ok=True)

cmd_list = []
for silent in glob.glob(f'{work_dir}validation_splites/*.silent'):
    name = silent.replace('.silent','')
    cmd = f'{PROTOCOL} -in:file:silent {silent} -extra_res_fa {param} -out:path:all {out} '+\
          f'-out:file:silent {name}_relax.silent'
    cmd_list.append(cmd)
fp = open(f'{work_dir}2_relax_all_cmd','w')
fp.write('\n'.join(cmd_list))
fp.close()

In [24]:
make_submit_file(cmds=f'{work_dir}2_relax_all_cmd',
    submitfile=f'{work_dir}2_relax_all_cmd'+'submit.sh',
    queue='short',
    group_size=1,
    logsfolder='logs',
    cpus=1,
    mem='1g',
    limit_job_to100=False,
    needs_gpu=False)